In [1]:

import plotly.express as px
import pandas as pd

import sys; sys.path.append('../..')
from src.results import read_results

trades = read_results.get_trades("solarsail-simplified")
# trades = read_results.get_trades("ApplesSellEODOptions")
print(len(list(trades)))

359


In [2]:
%load_ext dotenv
%dotenv

In [3]:
import datetime
import typing
import collections
import itertools
from src.backtest import exiters, execute_exits

tps = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
sls = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# sls = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
points = []
for sl in sls:
    rois_by_time = {}

    bracketed_trades = []
    for trade in trades:
        # print('    ', trade.get_start())
        exiter = exiters.StopLossTrailingPercentageExiter(sl)
        # t = execute_exits.find_exit_within_trade_timeframe(trade, exiter)
        t = execute_exits.find_exit_within_trade_day(trade, exiter)
        bracketed_trades.append(t)

    profit = sum(t.get_profit_loss() for t in bracketed_trades)
    print(sl, collections.Counter(t.is_win() for t in bracketed_trades), profit)


    for t in bracketed_trades:
        key = t.get_start().time()
        rois_by_time[key] = rois_by_time.get(key, []) + [t.get_profit_loss()]

    points.append({
        'sl': sl,
        'profit': profit,
        "rois_by_time": rois_by_time,
    })


0.05 Counter({False: 238, True: 121}) -15.431470000000097
0.1 Counter({False: 254, True: 105}) -15.79095999999994
0.2 Counter({False: 259, True: 100}) 22.123540000000077
0.3 Counter({False: 255, True: 104}) -22.266710000000074
0.4 Counter({False: 256, True: 103}) -35.04260000000001
0.5 Counter({False: 257, True: 102}) 16.34449999999996
0.6 Counter({False: 257, True: 102}) 16.396099999999954
0.7 Counter({False: 257, True: 102}) 16.196099999999966
0.8 Counter({False: 257, True: 102}) 16.196099999999966
0.9 Counter({False: 257, True: 102}) 16.196099999999966


In [4]:
def group_rois_by_time(rois_by_time):
    grouped_rois = {}
    for time, rois in rois_by_time.items():
        new_time = datetime.time(time.hour, time.minute // 15 * 15)
        grouped_rois[new_time] = grouped_rois.get(new_time, []) + rois
    return grouped_rois

for point in points:
    for time, rois in sorted(group_rois_by_time(point["rois_by_time"]).items()):
        print(time, len(rois), sum(rois) / len(rois))
    
    fig = px.histogram(pd.DataFrame([
        {
            'time': time,
            'freq': len(rois),
            'roi': sum(rois) / len(rois),
        }
        for time, rois in sorted(group_rois_by_time(point["rois_by_time"]).items())
    ]), x="time", y="roi", title=f"SL: {point['sl']:%}")
    fig.show()


04:30:00 2 -0.1455000000000004
04:45:00 1 0.13750000000000018
05:00:00 1 -0.10749999999999993
05:15:00 1 -0.09949999999999992
05:45:00 1 0.10399999999999965
08:15:00 1 -0.1054999999999997
08:45:00 1 0.3224999999999998
09:30:00 3 0.11276666666666657
09:45:00 10 0.06524999999999936
10:00:00 22 -0.13044318181818226
10:15:00 18 -0.042054722222222514
10:30:00 11 -0.19436363636363668
10:45:00 5 -0.12180000000000021
11:00:00 10 0.5003000000000003
11:15:00 12 -0.0246458333333337
11:30:00 13 0.05960423076923067
11:45:00 8 -0.2342712500000007
12:00:00 14 -0.2786532142857147
12:15:00 16 -0.05240625000000004
12:30:00 15 -0.35639200000000015
12:45:00 5 0.01081499999999993
13:00:00 16 0.038324999999999665
13:15:00 13 -0.016033846153846253
13:30:00 19 -0.23653526315789514
13:45:00 15 -0.04803000000000015
14:00:00 15 -0.13110933333333305
14:15:00 14 0.22964285714285643
14:30:00 11 0.06395272727272679
14:45:00 9 0.10305499999999942
15:00:00 21 0.022066428571428147
15:15:00 18 0.020249999999999675
15:30

04:30:00 2 -0.36599999999999966
04:45:00 1 -0.004999999999999893
05:00:00 1 -0.23499999999999988
05:15:00 1 -0.21899999999999986
05:45:00 1 0.0860000000000003
08:15:00 1 -0.22099999999999964
08:45:00 1 -0.2649999999999988
09:30:00 3 0.36510000000000015
09:45:00 10 -0.33779999999999977
10:00:00 22 0.06036363636363656
10:15:00 18 -0.28884277777777734
10:30:00 11 -0.13672727272727267
10:45:00 5 -0.038199999999999346
11:00:00 10 0.11380000000000039
11:15:00 12 0.00029166666666607943
11:30:00 13 0.06251923076923077
11:45:00 8 -0.7277699999999994
12:00:00 14 -0.588450714285714
12:15:00 16 -0.11556249999999994
12:30:00 15 -0.6533373333333328
12:45:00 5 -0.05457000000000001
13:00:00 16 -0.03046312499999966
13:15:00 13 -0.07247692307692306
13:30:00 19 0.025498947368421315
13:45:00 15 -0.13077999999999984
14:00:00 15 -0.2646133333333327
14:15:00 14 1.9739285714285717
14:30:00 11 0.07291999999999983
14:45:00 9 0.2187888888888888
15:00:00 21 -0.08963666666666657
15:15:00 18 -0.07411111111111114
15

04:30:00 2 -0.8069999999999999
04:45:00 1 -0.13799999999999946
05:00:00 1 -0.48999999999999977
05:15:00 1 -0.258
05:45:00 1 -0.17799999999999994
08:15:00 1 0.4600000000000004
08:45:00 1 -1.4399999999999995
09:30:00 3 0.39243333333333336
09:45:00 10 -0.5541999999999998
10:00:00 22 1.3500227272727277
10:15:00 18 1.038303333333334
10:30:00 11 -0.1014545454545456
10:45:00 5 -0.6583999999999992
11:00:00 10 0.13079999999999964
11:15:00 12 -0.8179999999999992
11:30:00 13 0.03354000000000004
11:45:00 8 -0.36199000000000037
12:00:00 14 -0.24041571428571443
12:15:00 16 -0.043312499999999844
12:30:00 15 -0.5160866666666666
12:45:00 5 -0.17159600000000003
13:00:00 16 0.4305762500000003
13:15:00 13 -0.1589
13:30:00 19 -0.212056842105263
13:45:00 15 -0.4724133333333332
14:00:00 15 -0.4428266666666662
14:15:00 14 2.4985714285714296
14:30:00 11 0.2497672727272729
14:45:00 9 -0.1555888888888889
15:00:00 21 -0.4589190476190473
15:15:00 18 -0.06033333333333349
15:30:00 23 -0.11776956521739128
15:45:00 15

04:30:00 2 -0.5830000000000006
04:45:00 1 -0.44200000000000017
05:00:00 1 -0.29000000000000004
05:15:00 1 -0.4590000000000003
05:45:00 1 -0.3790000000000002
08:15:00 1 0.11500000000000021
08:45:00 1 -2.615
09:30:00 3 -0.005233333333333645
09:45:00 10 -1.1425000000000007
10:00:00 22 1.4224999999999997
10:15:00 18 1.1722272222222225
10:30:00 11 -0.3256363636363638
10:45:00 5 -0.8958000000000006
11:00:00 10 0.38189999999999974
11:15:00 12 -0.7990833333333344
11:30:00 13 0.08920153846153842
11:45:00 8 -1.4472100000000014
12:00:00 14 -0.5403428571428579
12:15:00 16 0.28876249999999987
12:30:00 15 -0.6259466666666668
12:45:00 5 -0.2015340000000001
13:00:00 16 0.09859562499999963
13:15:00 13 -0.13809230769230768
13:30:00 19 -0.37155789473684236
13:45:00 15 -0.6680133333333332
14:00:00 15 -0.6028933333333332
14:15:00 14 1.6249285714285713
14:30:00 11 0.20808181818181828
14:45:00 9 -0.3700333333333335
15:00:00 21 -0.5363666666666668
15:15:00 18 -0.10834444444444462
15:30:00 23 -0.10333478260869

04:30:00 2 -1.1190000000000007
04:45:00 1 -0.6499999999999999
05:00:00 1 -0.6099999999999999
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 -3.79
09:30:00 3 -0.11623333333333356
09:45:00 10 -1.3966000000000003
10:00:00 22 1.0307454545454542
10:15:00 18 1.222705555555556
10:30:00 11 -0.44200000000000017
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07469230769230767
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.3008249999999998
12:30:00 15 -0.6343466666666667
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.75268
14:00:00 15 -0.5845199999999998
14:15:00 14 1.0052857142857143
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:45:00 15 -0.

04:30:00 2 -1.2025000000000001
04:45:00 1 -0.36999999999999966
05:00:00 1 -0.3299999999999996
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 5.449999999999999
09:30:00 3 -0.4499
09:45:00 10 -1.6740000000000002
10:00:00 22 3.618413636363636
10:15:00 18 1.1760388888888893
10:30:00 11 -0.5395454545454546
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07059999999999998
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.29861249999999984
12:30:00 15 -0.6359466666666668
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.7540133333333332
14:00:00 15 -0.5845199999999998
14:15:00 14 0.3371428571428573
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:4

04:30:00 2 -1.035
04:45:00 1 -0.36999999999999966
05:00:00 1 -0.3299999999999996
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 5.449999999999999
09:30:00 3 -0.5160333333333332
09:45:00 10 -1.6440000000000001
10:00:00 22 3.557959090909091
10:15:00 18 1.1660388888888893
10:30:00 11 -0.5563636363636364
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07059999999999998
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.29861249999999984
12:30:00 15 -0.6359466666666668
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.7540133333333332
14:00:00 15 -0.5845199999999998
14:15:00 14 0.4307142857142861
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:45

04:30:00 2 -1.035
04:45:00 1 -0.36999999999999966
05:00:00 1 -0.3299999999999996
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 5.449999999999999
09:30:00 3 -0.5160333333333332
09:45:00 10 -1.6640000000000001
10:00:00 22 3.557959090909091
10:15:00 18 1.1660388888888893
10:30:00 11 -0.5563636363636364
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07059999999999998
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.29861249999999984
12:30:00 15 -0.6359466666666668
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.7540133333333332
14:00:00 15 -0.5845199999999998
14:15:00 14 0.4307142857142861
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:45

04:30:00 2 -1.035
04:45:00 1 -0.36999999999999966
05:00:00 1 -0.3299999999999996
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 5.449999999999999
09:30:00 3 -0.5160333333333332
09:45:00 10 -1.6640000000000001
10:00:00 22 3.557959090909091
10:15:00 18 1.1660388888888893
10:30:00 11 -0.5563636363636364
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07059999999999998
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.29861249999999984
12:30:00 15 -0.6359466666666668
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.7540133333333332
14:00:00 15 -0.5845199999999998
14:15:00 14 0.4307142857142861
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:45

04:30:00 2 -1.035
04:45:00 1 -0.36999999999999966
05:00:00 1 -0.3299999999999996
05:15:00 1 0.1200000000000001
05:45:00 1 0.20000000000000018
08:15:00 1 -0.11999999999999966
08:45:00 1 5.449999999999999
09:30:00 3 -0.5160333333333332
09:45:00 10 -1.6640000000000001
10:00:00 22 3.557959090909091
10:15:00 18 1.1660388888888893
10:30:00 11 -0.5563636363636364
10:45:00 5 -1.1720000000000002
11:00:00 10 0.29599999999999993
11:15:00 12 -0.619166666666667
11:30:00 13 0.07059999999999998
11:45:00 8 -1.3275000000000001
12:00:00 14 -0.4715357142857145
12:15:00 16 0.29861249999999984
12:30:00 15 -0.6359466666666668
12:45:00 5 -0.17404000000000008
13:00:00 16 0.4826999999999999
13:15:00 13 -0.13986153846153843
13:30:00 19 -0.3383473684210527
13:45:00 15 -0.7540133333333332
14:00:00 15 -0.5845199999999998
14:15:00 14 0.4307142857142861
14:30:00 11 0.3192636363636365
14:45:00 9 -0.21781111111111123
15:00:00 21 -0.5468428571428572
15:15:00 18 -0.1290666666666668
15:30:00 23 -0.10333478260869564
15:45

In [ ]:
assert False
# CODE not updated yet:

In [ ]:

fig = px.density_heatmap(pd.DataFrame(points), x="sl", y="tp", z="profit", nbinsx=len(sls), nbinsy=len(tps))
fig.show()

In [ ]:
for sl, tp in itertools.product(sls, tps):
    bracket_set= typing.cast(
        list[brackets_historical.Bracket],
        [{'take_profit_percentage': tp, 'stop_loss_percentage': sl, 'until': datetime.time(15, 59)}]
    )
    bracketed_trades = [brackets_historical.get_bracketed_virtual_trade(t, bracket_set) for t in trades]
    fig = px.histogram(
        pd.DataFrame([{'roi': t.get_profit_loss() / t.get_value_spent()} for t in bracketed_trades]),
        x='roi',
        title=f'Bracketed trades sl={sl} tp={tp}',
        marginal='rug',
        range_x=[-1, 1],
    )
    fig.show()
    print("^^^", sl, tp, collections.Counter(t.is_win() for t in bracketed_trades),
          sum(t.get_profit_loss() for t in bracketed_trades))